In [34]:
# Bari - Desafio de Estágio
# Mateus Luiz Bailo - Email: mateus_bailo@hotmail.com

# -------------------------------
# 1. Carregamento de Bibliotecas
# -------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sqlalchemy import create_engine
from dotenv import load_dotenv


In [50]:
# --------------------------
# 2. Conexão com o Banco de Dados MySQL
# --------------------------

load_dotenv()

usuario = os.getenv("DB_USER")
senha = os.getenv("DB_PASSWORD").replace("@", "%40")
host = os.getenv("DB_HOST")
porta = os.getenv("DB_PORT")
banco = os.getenv("DB_NAME")



conexao = create_engine(f"mysql+mysqlconnector://{usuario}:{senha}@{host}:{porta}/{banco}")

In [ ]:
# --------------------------
# 3. Consulta SQL: Carregamento dos Dados
# --------------------------

query = """
    SELECT id_cliente as IdCliente
          ,idade AS Idade
          ,renda_mensal as RendaMensal
          ,score_credito as ScoreCredito
          ,valor_imovel as ValorImovel
          ,prazo_contrato_meses as ContratoMeses
          ,valor_financiado as ValorFinanciado
          ,inadimplente as Inadimplente
    FROM clientes_credito limit 10
"""

df_sql = pd.read_sql(query, conexao)
print(df_sql.head())

   id_cliente  Idade  RendaMensal  ScoreCredito  ValorImovel  ContratoMeses  \
0           1     59     13921.40           548    463771.95            360   
1           2     49      6939.90           825    444990.02            360   
2           3     35      7116.43           761    489861.78            300   
3           4     63      2933.34           312    263163.69            300   
4           5     28      9427.98           683    419948.53            120   

   ValorFinanciado  Inadimplente  
0        255471.75             0  
1        194613.60             0  
2        305831.89             0  
3        173892.50             1  
4        307163.49             0  


In [51]:
# --------------------------
# 4. Análise Exploratória
# --------------------------

print(df_sql.info())
print(df_sql.describe())
print(df_sql.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id_cliente            10 non-null     int64  
 1   idade                 10 non-null     int64  
 2   renda_mensal          10 non-null     float64
 3   score_credito         10 non-null     int64  
 4   valor_imovel          10 non-null     float64
 5   prazo_contrato_meses  10 non-null     int64  
 6   valor_financiado      10 non-null     float64
 7   inadimplente          10 non-null     int64  
dtypes: float64(3), int64(5)
memory usage: 772.0 bytes
None
       id_cliente      idade  renda_mensal  score_credito   valor_imovel  \
count    10.00000  10.000000     10.000000      10.000000      10.000000   
mean      5.50000  44.700000   7938.313000     628.900000  400565.019000   
std       3.02765  12.347289   3113.855269     182.932562   70663.988518   
min       1.00000  28.000

In [57]:
# --------------------------
# 4. Análise Exploratória
# --------------------------

query_risco = """
SELECT idade as Idade
      ,renda_mensal as RendaMensal
      ,score_credito as ScoreCredito
      ,inadimplente as Inadimplente
FROM clientes_credito
WHERE score_credito < 500 AND inadimplente = 1
"""

df_risco = pd.read_sql(query_risco, conexao)
print("Clientes com alto risco:")
print(df_risco.head())

Clientes com alto risco:
   Idade  RendaMensal  ScoreCredito  Inadimplente
0     63      2933.34           312             1
1     22      8071.49           463             1
2     36      4592.40           493             1
3     28      3934.46           303             1
4     67     12291.91           392             1
